### Guided Project: Analyzing CIA Fcatbook Data Using SQL
#### *By Naftali N Indongo*

### 1. Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
-  <font color = red>population</font> — the global population
-  <font color = red>population_growth</font> — the annual population growth rate, as a percentage.
- <font color = red>area</font> — the total land and water area.

In this guided project, we'll use SQL in Jupyter Notebook to analyze data from this database.

We'll use the following code to connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

### 2. Overview of the Data

We begin by getting an understanding on how our dataset looks like. This can be done by querying the database to get the information about the table.

In [2]:
%%sql
SELECT *
  FROM sqlite_master
WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Below we will get an overview of the data by looking at the first five rows of the <font color=red>facts</font> table in the database.

In [3]:
%%sql
SELECT *
  FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### 3. Summary Statistics

Here are the descriptions for some of the columns:

-  <font color = red>code</font> — the code of the country.
-  <font color = red>name</font> — the name of the country.
-  <font color = red>area</font> — the country's total area (both land and water).
-  <font color = red>area_land</font> — the country's land area in [square kilometers](https://www.cia.gov/library/publications/the-world-factbook/rankorder/2147rank.html).
-  <font color = red>area_water</font> — the country's waterarea in square kilometers.
-  <font color = red>population</font> — the country's population.
-  <font color = red>population_growth</font> — the country's population growth as a percentage.
-  <font color = red>birth_rate</font> — the country's birth rate, or the number of births per year per 1,000 people.
-  <font color = red>death_rate</font> — the country's death rate, or the number of death per year per 1,000 people.

We will start by calculating some summary statistics and look for any outlier countries.

In [4]:
%%sql
SELECT MIN(population) as min_pop, MAX(population) as max_pop, MIN(population_growth) as min_pop_growth, MAX(population_growth) as max_pop_growth
  FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


### 4. Exploring Outliers

From the summary statistics table we computed above we observe a few interesting things.

-  There is a country with a population of <font color=red>0</font>. 
-  There's a country with a population of <font color=red>7256490011</font> (or more than 7.2 billion people)

We will use subqueries to zoom in on these countries without using the specific values.

In [5]:
%%sql
SELECT *
  FROM facts
WHERE population = (SELECT MIN(population)
                      FROM facts
                   );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It seems like the table contains a row for Antarctica, which explains the population of <font color =red>0</font>. This seems to match the [CIA Factbook page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html):

![Info Artactica](https://camo.githubusercontent.com/6e15f6abc5b22dd52b087ba616738d5f6dd39fc666d445608f0d7c933892e5f9/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f64712d636f6e74656e742f3235372f66625f616e74617263746963612e706e67)

In [6]:
%%sql
SELECT *
  FROM facts
WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion.

Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.

### 5. Revised Summary Statistics

Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name != 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


There is a country with a population close to about 1.4 billion! We will further explore which country is that.

In [8]:
%%sql
SELECT *
  FROM facts
WHERE population == (SELECT MAX(population)
                       FROM facts
                      WHERE name != 'World'
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


$\therefore$ The country with the highest population is china.

### 6. Exploring Average Population and Area

Now we will explore population density. The country's population density depends on its population and area. We will calculate the average value for the following columns:

-  <font color=red>population</font>
-  <font color=red>area</font>

We should consider discarding the row for the whole world.

In [9]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
WHERE name != 'World'

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


In [10]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
WHERE name != 'World'

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


We observe that the average population is around 32 million and the average area is around 555 thousand square kilometers.

### 7. Finding Densely populated Countries

To finish, we'll build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have the following:

-  Above-average values for population.
-  Below-average values for area.

Instead of copying the values we calculated on the previous screen, we will make use of subquerries to write this query!

In [12]:
%%sql
SELECT *
  FROM facts
WHERE population > (SELECT AVG(population)
                      FROM facts
                     WHERE name != 'World') AND
      area < (
              SELECT AVG(area)
                FROM facts
               WHERE name != 'World');
                

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Most of these countries are generally known to be densely populated, so we have confidence in our results!

### 8. Further Exploration

We will explore further by looking at the following 3 questions:
-  Which country has the highest growth rate?
-  Which countries have a higher death rate than birth rate?
-  Which countries have the highest <font color=red>population/area</font> ratio, and how does it compare to list we found in the previous screen?

#### a) Country With the Highest Growth Rate
Will use the following query to find the country with highest poplation growth rate.

In [23]:
%%sql
SELECT *
  FROM facts
WHERE population_growth = (SELECT MAX(population_growth)
                             FROM facts
                            WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


$\therefore$ The country with the highest population growth is South Sudan.

#### b) Exploring Countries by Birth Rate and Death Rate

Here will explore which countries have higher birth rate than death and vice versa.

In [46]:
%%sql
SELECT *
  FROM facts
WHERE birth_rate < death_rate

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33


In [47]:
%%sql
SELECT count()
  FROM facts
WHERE birth_rate < death_rate

Done.


count()
24


$\therefore$ We have 24 countries with the higher death rate than birth rate.

#### c) Exploring the Ratio of Populatrion to Area

Here will explore which countries have the highest population/area ratio of water to land

In [36]:
%%sql
SELECT *
  FROM facts
WHERE population/area = (SELECT MAX(population/area) as pio
                           FROM facts
                           WHERE name != 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37


$\therefore$ The country with highest <font color=red>population/area</font> is Macau.